In [1]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF

In [2]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict

experimentlist()

['midHolocene-cal-adj',
 'atl3',
 'lig127k',
 'midHolocene',
 'barchart',
 'lig127-cal-adj',
 'HadISST',
 'piControl',
 'gpcp',
 'amm',
 'historical',
 '1pctCO2',
 'abrupt4xCO2',
 'midPliocene-eoi400',
 'reanalysis',
 'observations',
 'lgm',
 'lgm-cal-adj']

In [6]:
HadISST= "../data/HadISST/ACCESS-ESM1-5_HadISST.cvdp_data.1870-2016.nc"
xr.open_dataset(HadISST,decode_times=False), 

(<xarray.Dataset>
 Dimensions:                       (TIME: 147, frequency: 882, hov_lon: 160, i: 360, i2: 360, j: 180, j2: 180, lat: 180, lon: 360, ncurves: 4, period: 130, time: 1764, time_mon1: 29, time_mon2: 12, time_mon3: 49)
 Coordinates:
   * TIME                          (TIME) int32 1870 1871 1872 ... 2014 2015 2016
   * frequency                     (frequency) float32 0.0005669 0.001134 ... 0.5
   * hov_lon                       (hov_lon) float32 120.5 121.5 ... 278.5 279.5
   * i                             (i) float32 -179.5 -178.5 ... 178.5 179.5
   * i2                            (i2) float32 -179.5 -178.5 ... 178.5 179.5
   * j                             (j) float32 -89.5 -88.5 -87.5 ... 88.5 89.5
   * j2                            (j2) float32 -89.5 -88.5 -87.5 ... 88.5 89.5
   * lat                           (lat) float32 -89.5 -88.5 -87.5 ... 88.5 89.5
   * lon                           (lon) float32 0.5 1.5 2.5 ... 358.5 359.5
   * ncurves                       (nc